In [6]:
import sys
import json
sys.path.append('..')
from src.analysis.model_performances import *
import pandas as pd
EVERYTHING_POSTFIX = 'everything'

In [28]:
# def to_dfs(results_all, model_name, answer_type, prompt_type, subs, domains):
#     d_ramifications_pretty = {
#         WITH_RAMIFICATIONS : 'R',
#         WITHOUT_RAMIFICATIONS : 'No R'
#     }
#     d_answer_type_pretty = {
#         TRUE_FALSE_ANSWER_TYPE : 'T/F',
#         FREE_ANSWER_TYPE : 'Free'
#     }
#     
#     dfs = []
#     for plan_length in PLAN_LENGTHS:
#         index = []
#         data = []
#         for question_category in QUESTION_CATEGORIES + [ALL_QUESTION_CATEGORIES_KEY]:
#             for ramifications in RAMIFICATION_TYPES:
#                 index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
#                 data_columns = {}
#                 for domain in domains:
#                     data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
#                 data.append(data_columns)
#         dfs.append(pd.DataFrame(data, index = index))
#     return dfs

def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for question_category in QUESTION_CATEGORIES + [ALL_QUESTION_CATEGORIES_KEY]:
            for ramifications in RAMIFICATION_TYPES:
                index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
                data_columns = {}
                for domain in domains:
                    data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                data.append(data_columns)
    return pd.DataFrame(data, index = index)

In [29]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = F1_SCORE_KEY)
stats_all = collect_stats_all(tf_answer_type(score_key = F1_SCORE_KEY))

100%|██████████| 25920/25920 [00:00<00:00, 50069.94it/s]


In [30]:
model_name = 'gemini'#'gemma-2b-it'
prompt_type = 'few_shot_5'
subs = WITHOUT_RANDOM_SUB
domains = DOMAIN_NAMES + [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
# domains = [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
save_key = 'all'#'transport_vs_other'
df = to_df(stats_all, model_name, answer_type, prompt_type, subs, domains)

In [31]:
df

,blocksworld,depots,driverlog,goldminer,grippers,logistics,miconic,mystery,npuzzle,satellite,spanner,visitall,zenotravel,TRANSPORTATION,NON_TRANSPORTATION,ALL_DOMAINS
"(1, object_tracking, R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1, object_tracking, No R, T/F)",0.750000,0.747475,0.668831,0.674641,0.742857,0.775000,0.796296,0.658333,0.694268,0.693467,0.690476,0.790698,0.724138,0.730650,0.708812,0.719324
"(1, fluent_tracking, R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1, fluent_tracking, No R, T/F)",0.662963,0.716667,0.727642,0.721591,0.814286,0.761111,0.800000,0.683333,0.722222,0.663889,0.834375,0.821429,0.733333,0.750811,0.732466,0.741009
"(1, state_tracking, R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(19, hallucination, No R, T/F)",0.875000,0.860000,0.766667,0.850000,0.700000,0.790000,0.835714,0.790000,0.925000,0.778571,0.750000,0.783333,0.775000,0.787500,0.824359,0.805696
"(19, composite_questions, R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(19, composite_questions, No R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(19, ALL_CATEGORIES, R, T/F)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# latex_table = r'\midrule \n'.join(df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format) for df in dfs)
latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


latex_table_all = r"""
\begin{table*}[h!]
\begin{adjustbox}{width=\textwidth,center}
""" + latex_table + """
\end{adjustbox}
\caption{""" + f'{model_name.replace("_", " ")}, {prompt_type.replace("_", " ")}, {subs.replace("_", " ")}, {answer_type_ext.replace("_", " ")}' + """}
\end{table*}
"""
os.makedirs(os.path.join(STATISTICS_PATH, 'tables_2'), exist_ok=True)
with open(os.path.join(STATISTICS_PATH, 'tables_2', f'{model_name}.{prompt_type}.{answer_type_ext}.{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_62304/1674328619.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [26]:
latex_table_all

'\n\\begin{table*}[h!]\n\\begin{adjustbox}{width=\\textwidth,center}\n\\begin{tabular}{lrrrrrrrrrrrrrrrr}\n\\toprule\n{} &  blocksworld &  depots &  driverlog &  goldminer &  grippers &  logistics &  miconic &  mystery &  npuzzle &  satellite &  spanner &  visitall &  zenotravel &  TRANSPORTATION &  NON\\_TRANSPORTATION &  ALL\\_DOMAINS \\\\\n\\midrule\n(1, object\\_tracking, R, T/F)         &          NaN &     NaN &        NaN &        NaN &       NaN &        NaN &      NaN &      NaN &      NaN &        NaN &      NaN &       NaN &         NaN &             NaN &                 NaN &          NaN \\\\\n(1, object\\_tracking, No R, T/F)      &         0.75 &    0.75 &       0.67 &       0.67 &      0.74 &       0.78 &     0.80 &     0.66 &     0.69 &       0.69 &     0.69 &      0.79 &        0.72 &            0.73 &                0.71 &         0.72 \\\\\n(1, fluent\\_tracking, R, T/F)         &          NaN &     NaN &        NaN &        NaN &       NaN &        NaN &      NaN 

In [30]:
'asdsa'.replace('a', 'b')

'bsdsb'